In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [3]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [4]:
f = open('call_rate_test.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['날짜', '종가']
['2024-03-21\t3.5290\n', '2024-03-20\t3.5330\n', '2024-03-19\t3.5130\n', '2024-03-18\t3.4410\n', '2024-03-15\t3.4640\n']


In [5]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None
    min_difference = pd.Timedelta.max

    for date in df['날짜']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def up_down_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['날짜', '종가', 'up_down'])

    # '날짜' 열을 datetime 형식으로 변환
    df['날짜'] = pd.to_datetime(df['날짜'])

    for i, row in df.iterrows():
        time = row['날짜']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['날짜'].values:
            prev_row = df[df['날짜'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['날짜'] == nearest_date].iloc[0]

        if row['종가'] > prev_row['종가']:
            up_down = '매파'
        elif row['종가'] == prev_row['종가']:
            up_down = 0
        else:
            up_down = '비둘기파'

        df_ud.loc[i] = [time.date(), row['종가'], up_down]

    return df_ud

In [6]:
# # 예시 데이터
# df = pd.DataFrame({
#     '날짜': ['2024-03-21', '2024-03-22', '2024-03-23'],
#     '종가': [3.5290, 3.5900, 3.5800]
# })
df = pd.DataFrame(new_datas)

result = up_down_call(df)
print(result)

            날짜      종가 up_down
0   2024-03-21  3.5290    비둘기파
1   2024-03-20  3.5330    비둘기파
2   2024-03-19  3.5130    비둘기파
3   2024-03-18  3.4410    비둘기파
4   2024-03-15  3.4640    비둘기파
5   2024-03-14  3.4860    비둘기파
6   2024-03-13  3.4710    비둘기파
7   2024-03-12  3.4210    비둘기파
8   2024-03-11  3.4000    비둘기파
9   2024-03-08  3.5460    비둘기파
10  2024-03-07  3.3750    비둘기파
11  2024-03-06  3.4770    비둘기파
12  2024-03-05  3.2370    비둘기파
13  2024-03-04  3.3660    비둘기파
14  2024-02-29  3.4630    비둘기파
15  2024-02-28  3.4010    비둘기파
16  2024-02-27  3.4650    비둘기파
17  2024-02-26  3.5680      매파
18  2024-02-23  3.6770      매파
19  2024-02-22  3.6590      매파
20  2024-02-21  3.6250      매파
21  2024-02-20  3.5900      매파
22  2024-02-19  3.6340      매파
23  2024-02-16  3.6610      매파
24  2024-02-15  3.6820      매파
25  2024-02-14  3.6440      매파
26  2024-02-13  3.6200      매파
27  2024-02-08  3.6010      매파
28  2024-02-07  3.5180      매파
29  2024-02-06  3.5720      매파
30  2024-02-05  3.5730      매파
31  2024

In [9]:
final_df = pd.DataFrame(result)
final_df

,날짜,종가,up_down
0,2024-03-21,3.5290,비둘기파
1,2024-03-20,3.5330,비둘기파
2,2024-03-19,3.5130,비둘기파
3,2024-03-18,3.4410,비둘기파
4,2024-03-15,3.4640,비둘기파
5,2024-03-14,3.4860,비둘기파
6,2024-03-13,3.4710,비둘기파
7,2024-03-12,3.4210,비둘기파
8,2024-03-11,3.4000,비둘기파
9,2024-03-08,3.5460,비둘기파
